In [1]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
import pickle

In [2]:
# Конфигурация устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Определение класса модели (должен быть таким же, как при обучении)
class TextClassifier(nn.Module):
    def __init__(self, n_classes):
        super(TextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('DeepPavlov/rubert-base-cased', return_dict=False)
        self.dropout = nn.Dropout(p=0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        output = self.dropout(outputs[1])  # берем pooled output
        return self.classifier(output)

In [6]:
# Загрузка модели
n_classes = 12  # Задайте правильное количество классов, использованных при обучении
model = TextClassifier(n_classes);
model.load_state_dict(torch.load('model_CFO_sd.pth', map_location=device))
model.to(device)
model.eval();

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
# Загрузка LabelEncoder
with open('label_encoder.pkl', 'rb') as file:
    le = pickle.load(file)

In [8]:
# Загрузка токенизатора
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')


In [9]:
# Функция для выполнения предсказания
def predict_url(description):
    encoded = tokenizer.encode_plus(
        description,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        _, predicted_index = torch.max(outputs, dim=1)

    return le.inverse_transform([predicted_index.item()])[0]

In [20]:

# Ввод описания пользователем и вывод предсказанного URL
description = 'Технологическая компания BСSports, специализирующаяся на развитии технологий в спортивной индустрии, расширяет VR/AR подразделение и приглашает разработчиков для усиления команды. Основное направление работы - создание спортивных VR-игр на движках Unity и Unreal Engine.Ждем в команду целеустремленных разработчиков , горящих VR идеями и способных двигать индустрию вперед вместе с нами!Основная задача:Проектирование и реализация сетевой архитектуры для многопользовательских проектов на игровом движке Unity.Обязанности:Разработка сетевой логики;Проектирование, реализация, отладка и поддержка сетевых механик;Оптимизация проекта и повышение производительности;Рефакторинг имеющегося кода;Ежедневное участие в онлайн планерках, с целью устранения возникших проблем, а также помощи товарищам по команде;Участие в тестах разрабатываемого продукта.Требования:Опыт работы с Unity от 3х лет.Хорошее знание C#, ООП, SOLID.Опыт создания игровых механик.Опыт работы с мультиплеером.Понимание клиент-серверной архитектуры.Понимание сетевых протоколов.Наличие завершенных проектов или пет-проектов, которые можно продемонстрировать.Умение анализировать производительность и проводить оптимизацию.Понимание всех циклов разработки игрового продукта.Умение работать с системами контроля версий Git.Умение работать в команде.Нацеленность на результат, соблюдение дедлайнов.Технический английский, достаточный для чтения документации.Интерес к созданию VR проектов;Будет плюсом:Знание мультиплеерных движков: Photon, Mirror, NetCode;Опыт реализации собственных мультиплеерных движков;Проектирование БД и знание SQL;Опыт работы с Docker, Cubernets, CI / CD;Опыт работы с VR и сборка проектов для Meta Quest и Pico;Реализация подгрузки динамического контента. CDN.Условия:Работа над интересными проектами, двигающими VR индустрию вперед;Возможно официальное оформление;Молодой коллектив VR-энтузиастов с массой новейшего VR / AR оборудования.Своевременная оплата (дважды в месяц).Просторные офисы в Москве (Москва сити) и в Минске (БЦ “Футура”).'


In [21]:
predicted_url = predict_url(description)
print(f'Предсказанная ссылка: {predicted_url}')

Предсказанная ссылка: https://gb.ru/geek_university/developer/programmer/1c-developer
